# Integrating Keras & TensorFlow - The Keras workflow, expanded 
## TensorFlow Dev Summit 2017
### YouTube ID: UeheTiBJ0Io
### My source code: https://github.com/hcchengithub/Toy-Video-QA-problem-on-Keras

The code shown on the YouTube video is very not working. I have fixed all problems and now . . . don't know how to play yet.
This command line:

    jupyter nbconvert "experimenting the source code.ipynb"  --to script

converts this jupyter notebook into a .py normal python script file.

In [1]:
import tensorflow as tf

In [2]:
video = tf.keras.layers.Input(shape=(None,150,150,3))

In [3]:
cnn = tf.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    pooling='avg')

In [4]:
cnn.trainable = False

#### Fix a tf 1.4 bug before going on
If your tf version is 1.4 then you probably have the same problem that will happen to the next cell:

    %USERPROFILE%\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\keras\_impl\keras\layers\wrappers.py in _compute_output_shape(self, input_shape)
        191                                                  input_shape[2:])
        192     child_output_shape = self.layer._compute_output_shape(  # pylint: disable=protected-access
    --> 193         child_input_shape).as_list()
        194     timesteps = input_shape[1]
        195     return tensor_shape.TensorShape([child_output_shape[0], timesteps] +

    AttributeError: 'NoneType' object has no attribute 'as_list'

[Tensorflow issue#14054](https://github.com/tensorflow/tensorflow/issues/14054) has fixed the problem. You need to manually modify `%USERPROFILE%\AppData\Local\Programs\Python\Python36\Lib\site-packages\tensorflow\python\layers\base.py` a little to fix the bug. 


In [6]:
encoded_frames = tf.keras.layers.TimeDistributed(cnn)(video)

In [7]:
encoded_vid = tf.keras.layers.LSTM(256)(encoded_frames)

In [9]:
question = tf.keras.layers.Input(shape=[100],dtype='int32')  # not (100)

In [10]:
x = tf.keras.layers.Embedding(10000,256,mask_zero=True)(question)

In [11]:
encoded_q = tf.keras.layers.LSTM(128)(x)

In [12]:
x = tf.keras.layers.concatenate([encoded_vid,encoded_q])

In [13]:
x = tf.keras.layers.Dense(128,activation=tf.nn.relu)(x)

In [14]:
outputs = tf.keras.layers.Dense(1000)(x)

In [15]:
model = tf.keras.models.Model([video,question],outputs)

In [23]:
model.compile(optimizer='adam', loss='mean_absolute_percentage_error')

`model` compiled ok then what? A hint from YouTube comment says: They put a version of this model on the Keras docs. No sample data though (as below) Actually https://github.com/keras-team/keras > somewhere 

### [Video question answering model](https://keras.io/getting-started/functional-api-guide/#visual-question-answering-model)

Now that we have trained our image QA model, we can quickly turn it into a video QA model. With appropriate training, you will be able to show it a short video (e.g. 100-frame human action) and ask a natural language question about the video (e.g. "what sport is the boy playing?" -> "football").
    
    from keras.layers import TimeDistributed
    video_input = Input(shape=(100, 224, 224, 3))
    # This is our video encoded via the previously trained vision_model (weights are reused)
    encoded_frame_sequence = TimeDistributed(vision_model)(video_input)  # the output will be a sequence of vectors
    encoded_video = LSTM(256)(encoded_frame_sequence)  # the output will be a vector

    # This is a model-level representation of the question encoder, reusing the same weights as before:
    question_encoder = Model(inputs=question_input, outputs=encoded_question)

    # Let's use it to encode the question:
    video_question_input = Input(shape=(100,), dtype='int32')
    encoded_video_question = question_encoder(video_question_input)

    # And this is our video question answering model:
    merged = keras.layers.concatenate([encoded_video, encoded_video_question])
    output = Dense(1000, activation='softmax')(merged)
    video_qa_model = Model(inputs=[video_input, video_question_input], outputs=output)
